# Bolig data from Dataforsyningen

## DAGI

Pandas indstillinger

In [1]:
import pandas as pd
# Set pandas options to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Set pandas option to display the full content of each column
pd.set_option('display.max_colwidth', None)



Kommune koder i Danmark

In [31]:
# File path to csv file
path = r"C:\Users\viet-intel\boligpriser\data\input\kommunekoder.csv"

# Load data from csv into datafram
column_names = ['navn', 'kommunekode']
kommune_df = pd.read_csv(path, sep=',', encoding='latin-1', dtype=str, header=None, names=column_names, index_col=False)

# Display the first 5 rows of the dataframe
display(kommune_df.head(3))
display(kommune_df.tail(3))

,navn,kommunekode
0,Albertslund,165
1,Allerød,201
2,Assens,420
3,Ballerup,151
4,Billund,530


,navn,kommunekode
93,Vordingborg,390
94,Ærø,492
95,Aabenraa,580
96,Aalborg,851
97,Århus,751


Udvægelse af kommune koder i region hovedstaden

In [32]:
# List of kommune names in region hovedstaden
region_hovedstad_list = ["Albertslund", "Allerød", "Ballerup", "Bornholms Regionskommune", "Brøndby", 
                "Dragør", "Egedal", "Fredensborg", "Frederiksberg", "Frederikssund", "Furesø", 
                "Gentofte", "Gladsaxe", "Glostrup", "Gribskov", "Halsnæs", "Helsingør", "Herlev", 
                "Hillerød", "Hvidovre", "Høje-Taastrup", "Hørsholm", "Ishøj", "København", "Lyngby-Taarbæk", 
                "Rudersdal", "Rødovre", "Tårnby", "Vallensbæk"]

# Creating new dataframe with only kommune names in region hovedstaden
region_hovedstad_df = kommune_df[kommune_df['navn'].isin(region_hovedstad_list)]

# Creating a list of kommune codes in region hovedstaden
region_hovedstad_kommunekode_list = region_hovedstad_df['kommunekode'].tolist()

# print(region_hovedstad_kommunekode_list)
print(region_hovedstad_kommunekode_list)

['165', '201', '151', '153', '155', '240', '210', '147', '250', '190', '157', '159', '161', '270', '217', '163', '219', '167', '169', '223', '183', '101', '230', '175', '185', '187']


Hent alle adresser ud fra kommunekoder

In [ ]:
import requests
import pandas as pd

test_list = ['0165']#, '201', '151', '153', '155', '240', '210', '147', '250', '190', '157', '159', '161', '270', '217', '163', '219', '167', '169', '223', '183', '101', '230', '175', '185', '187']

# Define a function to make an API request to the Danish API for housing data
def housing_data_call(kommunekode):
    # URL to API
    url = f"https://api.dataforsyningen.dk/adresser?kommunekode={kommunekode}&format=csv"

    # Define a session with certificate verification disabled
    session = requests.Session()
    session.verify = False

    try:
        # Make the API request using the session
        response = session.get(url)
        response.raise_for_status()  # Make sure the API call was successful
    except requests.exceptions.RequestException as e:
        print(f"Error while fetching data for kommune {kommunekode}: {e}")
        return None

    # Read the response content as a DataFrame
    load_housing_adresses = pd.read_csv(response.url, low_memory=False, dtype={'col10': 'str'})
    return load_housing_adresses

# Empty list to store the dataframes
data_frame_list = []

# Loop through the kommune codes in the test_list and make an API request for each kommune to fetch the data
for kommune in test_list:
    kommune_data = housing_data_call(kommune)
    if kommune_data is not None:
        data_frame_list.append(kommune_data)

# Concatenate all the dataframes into one dataframe
all_kommune_df = pd.concat(data_frame_list, axis=0, ignore_index=True)

display(all_kommune_df.head())
print("length:", len(all_kommune_df))


Loading csv to sql

In [5]:
# define path
path = r"C:\Users\viet-intel\boligpriser\data\input\DAGI.csv"

# load data into dataframe
DAGI_df = pd.read_csv(path)

# print the first 5 rows of the dataframe
display(DAGI_df.head(1))

# print length of dataframe
print("length:", len(DAGI_df))

# print dtypes
print(DAGI_df.dtypes)


C:\Users\viet-intel\AppData\Local\Temp\ipykernel_22808\2309470723.py:5: DtypeWarning: Columns (10,14,71,73) have mixed types. Specify dtype option on import or set low_memory=False.
  DAGI_df = pd.read_csv(path)


,id,status,oprettet,ændret,vejkode,vejnavn,adresseringsvejnavn,husnr,etage,dør,supplerendebynavn,postnr,postnrnavn,stormodtagerpostnr,stormodtagerpostnrnavn,kommunekode,kommunenavn,ejerlavkode,ejerlavnavn,matrikelnr,esrejendomsnr,etrs89koordinat_øst,etrs89koordinat_nord,wgs84koordinat_bredde,wgs84koordinat_længde,nøjagtighed,kilde,tekniskstandard,tekstretning,ddkn_m100,ddkn_km1,ddkn_km10,adressepunktændringsdato,adgangsadresseid,adgangsadresse_status,adgangsadresse_oprettet,adgangsadresse_ændret,regionskode,regionsnavn,jordstykke_ejerlavnavn,kvhx,sognekode,sognenavn,politikredskode,politikredsnavn,retskredskode,retskredsnavn,opstillingskredskode,opstillingskredsnavn,zone,jordstykke_ejerlavkode,jordstykke_matrikelnr,jordstykke_esrejendomsnr,kvh,højde,adgangspunktid,vejpunkt_id,vejpunkt_kilde,vejpunkt_nøjagtighed,vejpunkt_tekniskstandard,vejpunkt_x,vejpunkt_y,afstemningsområdenummer,afstemningsområdenavn,brofast,supplerendebynavn_dagi_id,navngivenvej_id,menighedsrådsafstemningsområdenummer,menighedsrådsafstemningsområdenavn,vejpunkt_ændret,ikrafttrædelse,nedlagt,adgangsadresse_ikrafttrædelse,adgangsadresse_nedlagt,adgangsadresse_darstatus,darstatus,storkredsnummer,storkredsnavn,valglandsdelsbogstav,valglandsdelsnavn,landsdelsnuts3,landsdelsnavn,betegnelse
0,ba8c261d-4341-41e2-8fc9-93a483db41c8,1,2016-04-14T08:43:55.757,2016-04-14T08:43:55.757,7043,Skolebakken,Skolebakken,26,NaN,NaN,NaN,8500,Grenaa,NaN,NaN,707,Norddjurs,2005853.0,"Bredstrup, Grenaa Jorder",6ah,49804.0,617441.92,6253889.63,56.414951,10.903634,A,5.0,TD,288.38,100m_62538_6174,1km_6253_617,10km_625_61,2016-04-28T12:06:39.230,490d7c11-11a7-4a8b-ac92-3084e298fb81,1,2016-04-14T08:43:55.757,2020-01-16T01:00:00.000,1082,Region Midtjylland,"Bredstrup, Grenaa Jorder",07077043__26_______,8231,Grenaa,1461,Østjyllands Politi,1169,Retten i Randers,66,Djurs,Byzone,2005853.0,6ah,49804.0,07077043__26,6.5,482863a2-f005-4ecd-8e07-e20b1b058fd6,2001b328-af45-11e7-847e-066cff24d637,Adressemyn,A,V0,10.902653,56.415582,4,Grenaa Idrætscenter,1.0,NaN,2c6508b3-330f-4ae4-9533-8c5f5934a993,16.0,Grenaa,2020-01-14T10:08:38.23155,2016-04-14T08:43:55.757,NaN,2016-04-14T08:43:55.757,NaN,3,3,8,Østjylland,C,Midtjylland-Nordjylland,DK042,Østjylland,"Skolebakken 26, 8500 Grenaa"


length: 3897491
id                      object
status                   int64
oprettet                object
ændret                  object
vejkode                  int64
                         ...  
valglandsdelsbogstav    object
valglandsdelsnavn       object
landsdelsnuts3          object
landsdelsnavn           object
betegnelse              object
Length: 83, dtype: object


In [ ]:
DAGI_df.dtypes

#### Connecting to local my-sql-server and creating a table

In [15]:
import mysql.connector

# Replace the following details with your actual credentials
host = 'localhost'
user = 'root'
password = '1234'
database = 'bolig'  # Replace 'bolig' with the name of your database

# SQL query to create the table
create_table_query = """
    CREATE TABLE IF NOT EXISTS DAGI (
        id VARCHAR(255),
        status INT,
        oprettet VARCHAR(255),
        ændret VARCHAR(255),
        vejkode INT,
        vejnavn VARCHAR(255),
        adresseringsvejnavn VARCHAR(255),
        husnr VARCHAR(255),
        etage VARCHAR(255),
        dør VARCHAR(255),
        supplerendebynavn VARCHAR(255),
        postnr INT,
        postnrnavn VARCHAR(255),
        stormodtagerpostnr FLOAT,
        stormodtagerpostnrnavn VARCHAR(255),
        kommunekode INT,
        kommunenavn VARCHAR(255),
        ejerlavkode FLOAT,
        ejerlavnavn VARCHAR(255),
        matrikelnr VARCHAR(255),
        esrejendomsnr FLOAT,
        etrs89koordinat_øst FLOAT,
        etrs89koordinat_nord FLOAT,
        wgs84koordinat_bredde FLOAT,
        wgs84koordinat_længde FLOAT,
        nøjagtighed VARCHAR(255),
        kilde FLOAT,
        tekniskstandard VARCHAR(255),
        tekstretning FLOAT,
        ddkn_m100 VARCHAR(255),
        ddkn_km1 VARCHAR(255),
        ddkn_km10 VARCHAR(255),
        adressepunktændringsdato VARCHAR(255),
        adgangsadresseid VARCHAR(255),
        adgangsadresse_status INT,
        adgangsadresse_oprettet VARCHAR(255),
        adgangsadresse_ændret VARCHAR(255),
        regionskode INT,
        regionsnavn VARCHAR(255),
        jordstykke_ejerlavnavn VARCHAR(255),
        kvhx VARCHAR(255),
        sognekode INT,
        sognenavn VARCHAR(255),
        politikredskode INT,
        politikredsnavn VARCHAR(255),
        retskredskode INT,
        retskredsnavn VARCHAR(255),
        opstillingskredskode INT,
        opstillingskredsnavn VARCHAR(255),
        zone VARCHAR(255),
        jordstykke_ejerlavkode FLOAT,
        jordstykke_matrikelnr VARCHAR(255),
        jordstykke_esrejendomsnr FLOAT,
        kvh VARCHAR(255),
        højde FLOAT,
        adgangspunktid VARCHAR(255),
        vejpunkt_id VARCHAR(255),
        vejpunkt_kilde VARCHAR(255),
        vejpunkt_nøjagtighed VARCHAR(255),
        vejpunkt_tekniskstandard VARCHAR(255),
        vejpunkt_x FLOAT,
        vejpunkt_y FLOAT,
        afstemningsområdenummer INT,
        afstemningsområdenavn VARCHAR(255),
        brofast FLOAT,
        supplerendebynavn_dagi_id FLOAT,
        navngivenvej_id VARCHAR(255),
        menighedsrådsafstemningsområdenummer FLOAT,
        menighedsrådsafstemningsområdenavn VARCHAR(255),
        vejpunkt_ændret VARCHAR(255),
        ikrafttrædelse VARCHAR(255),
        nedlagt VARCHAR(255),
        adgangsadresse_ikrafttrædelse VARCHAR(255),
        adgangsadresse_nedlagt VARCHAR(255),
        adgangsadresse_darstatus INT,
        darstatus INT,
        storkredsnummer INT,
        storkredsnavn VARCHAR(255),
        valglandsdelsbogstav VARCHAR(255),
        valglandsdelsnavn VARCHAR(255),
        landsdelsnuts3 VARCHAR(255),
        landsdelsnavn VARCHAR(255),
        betegnelse VARCHAR(255)
    );

"""

# Establish the connection and create the table
try:
    connection = mysql.connector.connect(host=host, user=user, password=password, database=database)
    if connection.is_connected():
        print("Connected to MySQL database")
        cursor = connection.cursor()
        cursor.execute(create_table_query)
        print("Table DAGI created successfully")
        cursor.close()
    else:
        print("Failed to connect to MySQL database")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the connection when done
    if 'connection' in locals():
        connection.close()
        print("Connection closed.")


Connected to MySQL database
Table DAGI created successfully
Connection closed.


#### Loading dataframe into sql database

In [23]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine

# Replace the following details with your actual credentials
host = 'localhost'
user = 'root'
password = '1234'
database = 'bolig'

# Define the chunk size
chunk_size = 100
# Establish the connection
try:
    connection = mysql.connector.connect(host=host, user=user, password=password, database=database, connection_timeout=5000)
    if connection.is_connected():
        print("Connected to MySQL database")

        # Create an SQLAlchemy engine
        engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

        # Calculate the number of chunks needed
        num_chunks = len(DAGI_df) // chunk_size + 1

        # Load data into the database in chunks using the Pandas to_sql() method
        table_name = 'dagi'
        for i in range(num_chunks):
            start_idx = i * chunk_size
            end_idx = (i + 1) * chunk_size
            chunk = DAGI_df.iloc[start_idx:end_idx]
            chunk.to_sql(name=table_name, con=engine, if_exists='append', index=False)
            print(f"Chunk {i + 1}/{num_chunks} uploaded successfully")

        print("Data loaded into the table in chunks successfully")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the connection when done
    if 'connection' in locals():
        connection.close()
        print("Connection closed.")


Connected to MySQL database
Chunk 1/38975 uploaded successfully
Chunk 2/38975 uploaded successfully
Chunk 3/38975 uploaded successfully
Chunk 4/38975 uploaded successfully
Chunk 5/38975 uploaded successfully
Chunk 6/38975 uploaded successfully
Chunk 7/38975 uploaded successfully
Chunk 8/38975 uploaded successfully
Chunk 9/38975 uploaded successfully
Chunk 10/38975 uploaded successfully
Chunk 11/38975 uploaded successfully
Chunk 12/38975 uploaded successfully
Chunk 13/38975 uploaded successfully
Chunk 14/38975 uploaded successfully
Chunk 15/38975 uploaded successfully
Chunk 16/38975 uploaded successfully
Chunk 17/38975 uploaded successfully
Chunk 18/38975 uploaded successfully
Chunk 19/38975 uploaded successfully
Chunk 20/38975 uploaded successfully
Chunk 21/38975 uploaded successfully
Chunk 22/38975 uploaded successfully
Chunk 23/38975 uploaded successfully
Chunk 24/38975 uploaded successfully
Chunk 25/38975 uploaded successfully
Chunk 26/38975 uploaded successfully
Chunk 27/38975 uplo

### Land (Jordstykker), data collection

In [37]:
# Define a function to make an API request to the Danish API for land data
def land_data_call(kommunekode):
  
    # URL for API
    url = f"https://api.dataforsyningen.dk/jordstykker?kommunekode={kommunekode}&format=csv"

    # Make a GET request to the API and read the response as a DataFrame
    load_land_data = pd.read_csv(url, low_memory=False)
    return load_land_data

# Empty list to store the dataframes
land_data_frame_list = []

# Loop through the kommune codes in region hovedstaden and make an API request for each kommune and fetch the data
for kommune in region_hovedstad_kommunekode_list:
    land_data = land_data_call(kommune)
    land_data_frame_list.append(land_data)
    
# concatenate all the dataframes into one dataframe
all_land_df = pd.concat(land_data_frame_list, axis=0, ignore_index=True)

display(all_land_df.head())
print("Length:", len(all_land_df))


,ændret,geo_ændret,geo_version,bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax,visueltcenter_x,visueltcenter_y,ejerlavkode,matrikelnr,kommunekode,kommunenavn,sognekode,sognenavn,regionskode,regionsnavn,retskredskode,retskredsnavn,udvidet_esrejendomsnr,esrejendomsnr,sfeejendomsnr,bfenummer,ejerlavnavn,featureid,fælleslod,moderjordstykke,registreretareal,arealberegningsmetode,vejareal,vejarealberegningsmetode,vandarealberegningsmetode
0,2023-03-29T21:04:30.316Z,2015-02-27T20:24:00.545Z,1,12.347892,55.673141,12.350759,55.674584,12.348973,55.673985,20551,11am,165,Albertslund,7148,Herstedøster,1084,Region Hovedstaden,0,NaN,1.650057e+09,57236.0,1302759,1302759.0,"Herstedvester By, Herstedvester",999291,NaN,NaN,13141,o,3160,b,ukendt
1,2023-03-29T21:04:30.316Z,2015-02-27T20:24:00.545Z,1,12.341121,55.664756,12.342912,55.665700,12.341721,55.665352,20551,12s,165,Albertslund,7155,Herstedvester,1084,Region Hovedstaden,0,NaN,1.650059e+09,58771.0,9874836,9874836.0,"Herstedvester By, Herstedvester",2647884,NaN,NaN,2393,o,1182,b,ukendt
2,2023-03-29T21:04:30.316Z,2022-07-13T21:48:52.247Z,4,12.358818,55.658780,12.362671,55.660253,12.359989,55.659484,20551,15ak,165,Albertslund,9134,Opstandelseskirkens,1084,Region Hovedstaden,0,NaN,1.650066e+09,66019.0,100095931,100095931.0,"Herstedvester By, Herstedvester",100107370,NaN,NaN,13755,o,7020,b,ukendt
3,2023-05-09T21:46:11.428Z,2023-05-09T21:46:11.428Z,5,12.360630,55.670630,12.365362,55.674651,12.363326,55.673775,20652,10a,165,Albertslund,7148,Herstedøster,1084,Region Hovedstaden,0,NaN,1.650036e+09,36042.0,2126175,2126175.0,"Herstedøster By, Herstedøster",1001576,NaN,NaN,50472,k,314,b,incl
4,2023-05-09T21:46:11.428Z,2023-05-09T21:46:11.428Z,2,12.360788,55.671963,12.362603,55.673198,12.361809,55.672650,20652,10dn,165,Albertslund,7148,Herstedøster,1084,Region Hovedstaden,0,NaN,1.650066e+09,66024.0,100174931,100174931.0,"Herstedøster By, Herstedøster",100195210,NaN,1001576.0,10584,o,780,b,ukendt


Length: 346347


## BBR.info

Hente BBR info ud fra kommune kode 

In [ ]:
import requests
import pandas as pd

# Set the API URL
api_url = "https://services.datafordeler.dk/BBR/BBRPublic/1/rest/enhed"

# Function to fetch data
def fetch_data(kommunekode):
    # Set the parameters
    params = {
        "username": "ANLVUSSNAP",
        "password": "Essvietcc567!",
        "Format": "JSON",
        "Kommunekode": kommunekode,
        "pagesize": 9999999,  # Specify the maximum page size
        "page": 1  # Request the first page
    }

    # Fetch data from the API
    response = requests.get(api_url, params=params)

    # Check the status code and proceed with processing the response
    if response.status_code == 200:
        try:
            data = response.json()
            df = pd.DataFrame(data)
            return df

        except ValueError as e:
            print(f"Failed to decode JSON for kommunekode {kommunekode}.")
            return None

    else:
        print(f"Failed to fetch data for kommunekode {kommunekode} from the API. Status code: {response.status_code}")
        return None

# List of kommunekode values to iterate over
kommunekode_list = ['0165', '0201', '0151', '0153', '0155', '0240', '0210', '0147', '0250', '0190', '0157', '0159', '0161', '0270', '0217', '0163', '0219', '0167', '0169', '0223', '0183', '0101', '0230', '0175', '0185', '0187']


# Initialize an empty list to store the DataFrames
dataframes = []

# Fetch data for each kommunekode
for kommunekode in kommunekode_list:
    df = fetch_data(kommunekode)
    if df is not None:
        dataframes.append(df)

# Concatenate all the dataframes in the list
final_df = pd.concat(dataframes)

# Save the DataFrame to a pickle file
final_df.to_pickle(r"C:\Users\viet-intel\boligpriser\data\output\bbr_enhed.pkl")

# Display the first few rows and the total length of the final DataFrame
display(final_df.head())
print("Length:", len(final_df))


In [34]:
# Load pickle file
bbr_df = pd.read_pickle(r"C:\Users\viet-intel\boligpriser\data\output\bbr_enhed.pkl")

print(len(bbr_df))

bbr_df.head(3)

1165203


,datafordelerOpdateringstid,adresseIdentificerer,enh020EnhedensAnvendelse,enh023Boligtype,enh026EnhedensSamledeAreal,enh027ArealTilBeboelse,enh030KildeTilEnhedensArealer,enh031AntalVærelser,enh032Toiletforhold,enh033Badeforhold,enh034Køkkenforhold,enh045Udlejningsforhold,enh048GodkendtTomBolig,enh065AntalVandskylledeToiletter,enh066AntalBadeværelser,enh071AdresseFunktion,etage,forretningshændelse,forretningsområde,forretningsproces,id_lokalId,id_namespace,kommunekode,opgang,registreringFra,registreringsaktør,status,virkningFra,virkningsaktør,bygning,enh024KondemneretBoligenhed,enh067Støjisolering,enh025OprettelsesdatoForEnhedensIdentifikation,enh035Energiforsyning,ejerlejlighedList,enh028ArealTilErhverv,enh046OffentligStøtte,enh063AntalVærelserTilErhverv,enh101Gyldighedsdato,enh060EnhedensAndelFællesAdgangsareal,enh070ÅbenAltanTagterrasseAreal,enh051Varmeinstallation,enh068FlexboligTilladelsesart,enh127FysiskArealTilBeboelse,fordelingsarealList,enh053SupplerendeVarme,enh062ArealAfLukketOverdækningUdestue,enh052Opvarmningsmiddel,enh039AndetAreal,enh047IndflytningDato,enh041LovligAnvendelse,enh044DatoForDelvisIbrugtagningsTilladelse,enh042DatoForTidsbegrænsetDispensation,enh008UUIDTilModerlejlighed,enh128FysiskArealTilErhverv
0,2021-04-03T17:52:53.115133+02:00,0a3f50a5-3c61-32b8-e044-0003ba298018,130,1,106.0,106.0,1,4.0,T,V,E,1,0,1.0,1.0,0,c0034a30-e3a9-47db-8df8-fc09ac978e06,Enhed,54.15.05.05,0,000146fc-eb6d-449e-9ecd-a90bea4b6963,http://data.gov.dk/bbr/enhed,0165,01e09215-259d-4a33-a16c-545e8300059c,2018-09-14T19:31:21.793333+02:00,BBR,6,2018-09-14T19:31:21.793333+02:00,BBR,643e4ffe-60ce-40d4-aec3-7f8e93709897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-27T15:07:49.672495+01:00,0a3f50a5-3b93-32b8-e044-0003ba298018,150,3,29.0,29.0,1,1.0,T,V,F,1,0,1.0,1.0,1,be4164f9-4c7c-4a62-8ca1-139a383656af,Enhed,54.15.05.05,4,00019635-4426-4b7f-9afc-9623facdc80f,http://data.gov.dk/bbr/enhed,0165,b46abcef-2997-4b1f-b108-05c2e7125a30,2023-02-27T15:07:24.284866+01:00,BBR,10,2023-02-27T15:07:24.284866+01:00,Registerfører,c5b3d260-7dd6-4179-a43b-2b267354383b,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-27T15:20:40.813256+01:00,0a3f50a5-3b5e-32b8-e044-0003ba298018,150,3,29.0,29.0,1,1.0,T,V,F,1,0,1.0,1.0,1,1aca0cd2-6782-436f-ada9-154aec8f4364,Enhed,54.15.05.05,4,0004658e-4916-4ef6-bc67-0973bde1ba24,http://data.gov.dk/bbr/enhed,0165,0dd607ca-5d1b-474e-8f70-63d4507948a6,2023-02-27T15:20:17.927211+01:00,BBR,10,2023-02-27T15:20:17.927211+01:00,Registerfører,f3cd9fa9-407b-47d2-8026-c99922b48038,0,1989.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
columns = ['datafordelerOpdateringstid', 'adresseIdentificerer', 'enh020EnhedensAnvendelse', 'enh023Boligtype', 
           'enh026EnhedensSamledeAreal', 'enh027ArealTilBeboelse', 'enh030KildeTilEnhedensArealer', 'enh031AntalVærelser', 
           'enh032Toiletforhold', 'enh033Badeforhold', 'enh034Køkkenforhold', 'enh045Udlejningsforhold', 
           'enh048GodkendtTomBolig', 'enh065AntalVandskylledeToiletter', 'enh066AntalBadeværelser', 'enh071AdresseFunktion', 
           'etage', 'forretningshændelse', 'forretningsområde', 'forretningsproces', 'id_lokalId', 'id_namespace', 
           'kommunekode', 'opgang', 'registreringFra', 'registreringsaktør', 'status', 'virkningFra', 'virkningsaktør', 
           'bygning', 'enh024KondemneretBoligenhed', 'enh067Støjisolering']

# Create new DataFrame with the selected columns
bbr_df = bbr_df[columns]

bbr_df.head(3)


,datafordelerOpdateringstid,adresseIdentificerer,enh020EnhedensAnvendelse,enh023Boligtype,enh026EnhedensSamledeAreal,enh027ArealTilBeboelse,enh030KildeTilEnhedensArealer,enh031AntalVærelser,enh032Toiletforhold,enh033Badeforhold,enh034Køkkenforhold,enh045Udlejningsforhold,enh048GodkendtTomBolig,enh065AntalVandskylledeToiletter,enh066AntalBadeværelser,enh071AdresseFunktion,etage,forretningshændelse,forretningsområde,forretningsproces,id_lokalId,id_namespace,kommunekode,opgang,registreringFra,registreringsaktør,status,virkningFra,virkningsaktør,bygning,enh024KondemneretBoligenhed,enh067Støjisolering
0,2021-04-03T17:52:53.115133+02:00,0a3f50a5-3c61-32b8-e044-0003ba298018,130,1,106.0,106.0,1,4.0,T,V,E,1,0,1.0,1.0,0,c0034a30-e3a9-47db-8df8-fc09ac978e06,Enhed,54.15.05.05,0,000146fc-eb6d-449e-9ecd-a90bea4b6963,http://data.gov.dk/bbr/enhed,0165,01e09215-259d-4a33-a16c-545e8300059c,2018-09-14T19:31:21.793333+02:00,BBR,6,2018-09-14T19:31:21.793333+02:00,BBR,643e4ffe-60ce-40d4-aec3-7f8e93709897,NaN,NaN
1,2023-02-27T15:07:49.672495+01:00,0a3f50a5-3b93-32b8-e044-0003ba298018,150,3,29.0,29.0,1,1.0,T,V,F,1,0,1.0,1.0,1,be4164f9-4c7c-4a62-8ca1-139a383656af,Enhed,54.15.05.05,4,00019635-4426-4b7f-9afc-9623facdc80f,http://data.gov.dk/bbr/enhed,0165,b46abcef-2997-4b1f-b108-05c2e7125a30,2023-02-27T15:07:24.284866+01:00,BBR,10,2023-02-27T15:07:24.284866+01:00,Registerfører,c5b3d260-7dd6-4179-a43b-2b267354383b,0,NaN
2,2023-02-27T15:20:40.813256+01:00,0a3f50a5-3b5e-32b8-e044-0003ba298018,150,3,29.0,29.0,1,1.0,T,V,F,1,0,1.0,1.0,1,1aca0cd2-6782-436f-ada9-154aec8f4364,Enhed,54.15.05.05,4,0004658e-4916-4ef6-bc67-0973bde1ba24,http://data.gov.dk/bbr/enhed,0165,0dd607ca-5d1b-474e-8f70-63d4507948a6,2023-02-27T15:20:17.927211+01:00,BBR,10,2023-02-27T15:20:17.927211+01:00,Registerfører,f3cd9fa9-407b-47d2-8026-c99922b48038,0,1989.0


Load til SQL

In [ ]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, types
from math import ceil

# MySQL connection setup
username = 'root'
password = '1234'
host = 'localhost'
database = 'bolig'

engine_str = f"mysql+mysqlconnector://{username}:{password}@{host}/{database}"
engine = create_engine(engine_str)

# Fetching the dtypes for each column
dtype_dict = bbr_df.dtypes.apply(lambda x: types.TEXT 
                                  if x == "object" 
                                  else types.Float(precision=3, asdecimal=True) 
                                  if x == "float" 
                                  else types.Integer()).to_dict()

# Create table if doesn't exist
bbr_df[:0].to_sql('bbr', con=engine, if_exists='replace', dtype=dtype_dict)

# Upload data in chunks
n = 1000  # chunk row size
total_chunks = ceil(len(bbr_df) / n)

for i in range(total_chunks):
    print(f'Uploading chunk {i+1}/{total_chunks}...')
    bbr_df[i*n:(i+1)*n].to_sql('bbr', con=engine, if_exists='append', dtype=dtype_dict)
    
print("Upload complete.")


## DAR

DAR information ud fra kommune kode

In [3]:
import os
import requests
import pandas as pd
import time

# Set the API URL
api_url = "https://services.datafordeler.dk/DAR/DAR/2.0.0/rest/adresse"

# Function to fetch data
def fetch_data(kommunekode):
    # Set the parameters
    params = {
        "username": "ANLVUSSNAP",
        "password": "Essvietcc567!",
        "Format": "JSON",
        "Kommunekode": kommunekode,
        "pagesize": 100000,  # Specify a reasonable page size
    }
    
    # Initialize an empty list to store the DataFrames
    dataframes = []
    
    # Initialize the page
    page = 1

    while True:
        # Update the 'page' parameter
        params["page"] = page

        # Fetch data from the API
        response = requests.get(api_url, params=params)
        time.sleep(1)  # Sleep for 1 second to prevent hitting rate limit

        # Check the status code and proceed with processing the response
        if response.status_code == 200:
            try:
                data = response.json()
                if not data:  # Break the loop if the data list is empty
                    break
                df = pd.DataFrame(data)
                dataframes.append(df)

            except ValueError as e:
                print(f"Failed to decode JSON for kommunekode {kommunekode}.")
                return None

        else:
            print(f"Failed to fetch data for kommunekode {kommunekode} from the API. Status code: {response.status_code}")
            return None
        
        # Increment the page number
        page += 1
    
    # Concatenate all dataframes
    if dataframes:
        return pd.concat(dataframes)
    else:
        return None

# List of kommunekode values to iterate over
kommunekode_list = ['0101'] 

# Output directory
output_dir = r'C:\Users\viet-intel\boligpriser\data\output\Hovedstad'

# Fetch data for each kommunekode
for kommunekode in kommunekode_list:
    df = fetch_data(kommunekode)
    if df is not None:
        # Print the length of the DataFrame
        print(f'Length of DataFrame for kommunekode {kommunekode}: {len(df)}')

        # Save the DataFrame to a pickle file
        pickle_file = os.path.join(output_dir, f'kommune_{kommunekode}.pkl')
        df.to_pickle(pickle_file)

print("Data saved to individual pickle files.")


Length of DataFrame for kommunekode 0101: 464484
Data saved to individual pickle files.


Sammensætning af forskellige pkl filer

In [2]:
import glob
import pandas as pd
import os

# Directory containing the pickle files
input_dir = r'C:\Users\viet-intel\boligpriser\data\output\Hovedstad'

# Output directory for the combined pickle file
output_dir = r'C:\Users\viet-intel\boligpriser\data\output'

# Get a list of all pickle files in the directory
pickle_files = glob.glob(os.path.join(input_dir, '*.pkl'))

# Load the data from each pickle file and store it in a list
dataframes = [pd.read_pickle(pickle_file) for pickle_file in pickle_files]

# Concatenate all dataframes into one
dar_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new pickle file
output_file = os.path.join(output_dir, 'DAR.pkl')
dar_df.to_pickle(output_file)

# Print the length of the combined DataFrame
print(f'Length of combined DataFrame: {len(dar_df)}')

print(f"Combined data saved to {output_file}.")



Length of combined DataFrame: 1144796
Combined data saved to C:\Users\viet-intel\boligpriser\data\output\DAR.pkl.


In [4]:
dar_df.drop(columns=['husnummer'], inplace=True)

In [6]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, types
from math import ceil

# MySQL connection setup
username = 'root'
password = '1234'
host = 'localhost'
database = 'bolig'

engine_str = f"mysql+mysqlconnector://{username}:{password}@{host}/{database}"
engine = create_engine(engine_str)

# Fetching the dtypes for each column
dtype_dict = dar_df.dtypes.map(lambda x: types.TEXT 
                                  if x == "object" 
                                  else types.Float(precision=3, asdecimal=True) 
                                  if x == "float" 
                                  else types.Integer()).to_dict()

# Create table if doesn't exist
dar_df[:0].to_sql('dar', con=engine, if_exists='replace', dtype=dtype_dict)

# Upload data in chunks
n = 1000  # chunk row size
total_chunks = ceil(len(dar_df) / n)

for i in range(total_chunks):
    print(f'Uploading chunk {i+1}/{total_chunks}...')
    dar_df[i*n:(i+1)*n].to_sql('dar', con=engine, if_exists='append', dtype=dtype_dict)
    
print("Upload complete.")


Uploading chunk 1/1145...
Uploading chunk 2/1145...
Uploading chunk 3/1145...
Uploading chunk 4/1145...
Uploading chunk 5/1145...
Uploading chunk 6/1145...
Uploading chunk 7/1145...
Uploading chunk 8/1145...
Uploading chunk 9/1145...
Uploading chunk 10/1145...
Uploading chunk 11/1145...
Uploading chunk 12/1145...
Uploading chunk 13/1145...
Uploading chunk 14/1145...
Uploading chunk 15/1145...
Uploading chunk 16/1145...
Uploading chunk 17/1145...
Uploading chunk 18/1145...
Uploading chunk 19/1145...
Uploading chunk 20/1145...
Uploading chunk 21/1145...
Uploading chunk 22/1145...
Uploading chunk 23/1145...
Uploading chunk 24/1145...
Uploading chunk 25/1145...
Uploading chunk 26/1145...
Uploading chunk 27/1145...
Uploading chunk 28/1145...
Uploading chunk 29/1145...
Uploading chunk 30/1145...
Uploading chunk 31/1145...
Uploading chunk 32/1145...
Uploading chunk 33/1145...
Uploading chunk 34/1145...
Uploading chunk 35/1145...
Uploading chunk 36/1145...
Uploading chunk 37/1145...
Uploading 

## EBR

Enkel søgning

In [7]:
# Define the API endpoint URL
url = "https://services.datafordeler.dk/DAR/DAR_BFE_Public/1/rest/adresseTilEnhedBfe?username=ANLVUSSNAP&password=Essvietcc567!&Format=JSON&adresseId=0a3f509e-d15b-32b8-e044-0003ba298018"

try:
    # Make the API call and get the JSON response
    response = requests.get(url)
    response.raise_for_status()  # Check for any errors in the API call

    # Print the JSON response
    print(response.json())

except requests.exceptions.RequestException as e:
    print("Error making the API call:", e)
except ValueError as ve:
    print("Error processing JSON response:", ve)


[]


Load pkl

In [1]:
import pandas as pd

# Load pickle file into dataframe
path = r"C:\Users\viet-intel\boligpriser\data\output\DAR.pkl"

DAR = pd.read_pickle(path)

print(list(DAR.columns))
print(len(DAR))

Lav ny dataframe med id og tom bfe kolonne

In [ ]:
# select the columns id
bfe_id = DAR[['id_lokalId']]

# adding empty column bfe to the dataframe
bfe_id['BFE'] = ""

display(bfe_id.head())
len(bfe_id)

Finder tilsvarende BFE nummer til hver adresse

In [ ]:
import requests
import pandas as pd

def get_data_from_api(adresse_ids):
    url = f"https://services.datafordeler.dk/EBR/Ejendomsbeliggenhed/1/rest/BFEnrAdresse?Adresseid={adresse_ids}&username=ANLVUSSNAP&password=Essvietcc567!"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Error making the API call:", e)
        return None
    except ValueError as ve:
        print("Error processing JSON response:", ve)
        return None

def chunker(seq, size):
    # Function to divide the list into chunks of specified size
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# Load the dataframe
# region_hovedstad_df = pd.read_csv('your_data.csv')

# Select the first 1000 rows of the dataframe
adresse_ids = DAR['id_lokalId'].iloc[:100000].str.strip().tolist()

df_list = []
for chunk in chunker(adresse_ids, 20):
    adresse_ids_str = '|'.join(chunk)
    response_data = get_data_from_api(adresse_ids_str)
    if response_data and 'features' in response_data:
        for feature in response_data['features']:
            properties = feature['properties']
            df_list.append(properties)

# Convert the list of dictionaries to a DataFrame
bfe_df = pd.DataFrame(df_list)

# Display the first few rows of the DataFrame
display(bfe_df.head())
print(len(bfe_df))

# save the dataframe to a pickle file
bfe_df.to_pickle(r"C:\Users\viet-intel\boligpriser\data\output\EBR\bfe.pkl")

In [8]:
import pandas as pd
import os

# Get a list of all .pkl files in the directory
dir_path = r'C:\Users\viet-intel\boligpriser\data\output\EBR'
file_list = [f for f in os.listdir(dir_path) if f.endswith('.pkl')]

# Initialize an empty list that will be used to store the data frames
df_list = []

# Loop through the list of files and read each one into a pandas data frame
for file in file_list:
    file_path = os.path.join(dir_path, file)
    df = pd.read_pickle(file_path)  # load pickle file
    df_list.append(df)

# Concatenate all of the data frames into one
EBR_df = pd.concat(df_list, ignore_index=True)

# Print length of the combined DataFrame
print(f'Length of combined DataFrame: {len(EBR_df)}')

Length of combined DataFrame: 179463


In [9]:
EBR_df.head()


,id_namespace,id_lokalId,bestemtFastEjendomBFENr,Ejendomstype,adresseManueltAngivet,ESDHReferenceAdresse,ESDHReferenceKommune,kommuneManueltAngivet,kommuneinddelingKommunekode,betegnelse,...,status,forretningshaendelse,forretningsomraade,forretningsproces,virkningFra,virkningTil,virkningsaktoer,registreringFra,registreringTil,registreringsaktoer
0,http://data.gov.dk/Ejendomsbeliggenhedsregistret,fea8a5da-38ae-4504-afb6-a605d566e803,173846,Ejerlejlighed,False,None,None,False,0101,None,...,gældende,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:47.972423+02:00,None,Ejendomsbeliggenhedsregister
1,http://data.gov.dk/Ejendomsbeliggenhedsregistret,073e294d-7685-40ec-a802-87b2820983da,172353,Ejerlejlighed,False,None,None,False,0101,None,...,gældende,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:46.321174+02:00,None,Ejendomsbeliggenhedsregister
2,http://data.gov.dk/Ejendomsbeliggenhedsregistret,a4cdd92e-2fa0-4eb7-a599-68644e841806,172355,Ejerlejlighed,False,None,None,False,0101,None,...,gældende,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:46.321737+02:00,None,Ejendomsbeliggenhedsregister
3,http://data.gov.dk/Ejendomsbeliggenhedsregistret,2486fef4-13f9-4ee3-b78f-9785ea10d55d,172356,Ejerlejlighed,False,None,None,False,0101,None,...,gældende,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:46.322216+02:00,None,Ejendomsbeliggenhedsregister
4,http://data.gov.dk/Ejendomsbeliggenhedsregistret,2f57e964-cc7a-474c-a162-69161816ff50,172357,Ejerlejlighed,False,None,None,False,0101,None,...,gældende,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:46.322340+02:00,None,Ejendomsbeliggenhedsregister


Load to my SQL

In [10]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, types
from math import ceil

# MySQL connection setup
username = 'root'
password = '1234'
host = 'localhost'
database = 'bolig'

engine_str = f"mysql+mysqlconnector://{username}:{password}@{host}/{database}"
engine = create_engine(engine_str)

# Fetching the dtypes for each column
dtype_dict = EBR_df.dtypes.map(lambda x: types.TEXT 
                                  if x == "object" 
                                  else types.Float(precision=3, asdecimal=True) 
                                  if x == "float" 
                                  else types.Integer()).to_dict()

# Create table if doesn't exist
EBR_df[:0].to_sql('ebr', con=engine, if_exists='replace', dtype=dtype_dict)

# Upload data in chunks
n = 1000  # chunk row size
total_chunks = ceil(len(EBR_df) / n)

for i in range(total_chunks):
    print(f'Uploading chunk {i+1}/{total_chunks}...')
    EBR_df[i*n:(i+1)*n].to_sql('ebr', con=engine, if_exists='append', dtype=dtype_dict)
    
print("Upload complete.")


Uploading chunk 1/180...
Uploading chunk 2/180...
Uploading chunk 3/180...
Uploading chunk 4/180...
Uploading chunk 5/180...
Uploading chunk 6/180...
Uploading chunk 7/180...
Uploading chunk 8/180...
Uploading chunk 9/180...
Uploading chunk 10/180...
Uploading chunk 11/180...
Uploading chunk 12/180...
Uploading chunk 13/180...
Uploading chunk 14/180...
Uploading chunk 15/180...
Uploading chunk 16/180...
Uploading chunk 17/180...
Uploading chunk 18/180...
Uploading chunk 19/180...
Uploading chunk 20/180...
Uploading chunk 21/180...
Uploading chunk 22/180...
Uploading chunk 23/180...
Uploading chunk 24/180...
Uploading chunk 25/180...
Uploading chunk 26/180...
Uploading chunk 27/180...
Uploading chunk 28/180...
Uploading chunk 29/180...
Uploading chunk 30/180...
Uploading chunk 31/180...
Uploading chunk 32/180...
Uploading chunk 33/180...
Uploading chunk 34/180...
Uploading chunk 35/180...
Uploading chunk 36/180...
Uploading chunk 37/180...
Uploading chunk 38/180...
Uploading chunk 39/18

## Creating sample

In [12]:
import pandas as pd
from sqlalchemy import create_engine

# MySQL connection setup
username = 'root'
password = '1234'
host = 'localhost'
database = 'bolig'

# Establish a connection to the MySQL database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

# SQL query to select all data from the "ebr" table
query = 'SELECT * FROM ebr_sample WHERE ois = false'

# Load the data into a pandas DataFrame
ebrsql_df = pd.read_sql_query(query, engine)

# Close the database connection
engine.dispose()

# print length of dataframe
print(len(ebrsql_df))

72281


In [13]:
ebrsql_df.drop(columns=['part'])


,index,id_namespace,id_lokalId,bestemtFastEjendomBFENr,Ejendomstype,adresseManueltAngivet,ESDHReferenceAdresse,ESDHReferenceKommune,kommuneManueltAngivet,kommuneinddelingKommunekode,...,forretningshaendelse,forretningsomraade,forretningsproces,virkningFra,virkningTil,virkningsaktoer,registreringFra,registreringTil,registreringsaktoer,ois
0,9564,http://data.gov.dk/Ejendomsbeliggenhedsregistret,8dd528e4-2551-4c02-852e-8837fbdda1d0,414573,Ejerlejlighed,0,None,None,0,0101,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:22:51.482668+02:00,None,Ejendomsbeliggenhedsregister,0
1,9728,http://data.gov.dk/Ejendomsbeliggenhedsregistret,fb33c8c8-5bb7-4d4a-8b2f-588a7b69f53a,500287,Ejerlejlighed,0,None,None,0,0147,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:22:56.920166+02:00,None,Ejendomsbeliggenhedsregister,0
2,11488,http://data.gov.dk/Ejendomsbeliggenhedsregistret,2182e5a2-bc85-47f5-9f42-2801ea068c19,500181,Ejerlejlighed,0,None,None,0,0147,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:22:56.596559+02:00,None,Ejendomsbeliggenhedsregister,0
3,11729,http://data.gov.dk/Ejendomsbeliggenhedsregistret,c34ceed2-f96f-4b49-a93c-a3f226dfeffb,173521,Ejerlejlighed,0,None,None,0,0101,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:47.677911+02:00,None,Ejendomsbeliggenhedsregister,0
4,14107,http://data.gov.dk/Ejendomsbeliggenhedsregistret,8b8e1f14-c29a-488c-8d86-30ae3b387ccc,500276,Ejerlejlighed,0,None,None,0,0147,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:22:56.828305+02:00,None,Ejendomsbeliggenhedsregister,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72276,89728,http://data.gov.dk/Ejendomsbeliggenhedsregistret,26cfb8b6-8abb-4a0c-8546-35f53ab7eecc,161991,Ejerlejlighed,0,None,None,0,0101,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:44.629879+02:00,None,Ejendomsbeliggenhedsregister,0
72277,89729,http://data.gov.dk/Ejendomsbeliggenhedsregistret,f564f075-8bf9-4065-a8ee-a9a4a55c9524,119472,Ejerlejlighed,0,None,None,0,0101,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:33.996957+02:00,None,Ejendomsbeliggenhedsregister,0
72278,89730,http://data.gov.dk/Ejendomsbeliggenhedsregistret,da11882a-775c-4e19-93db-b079bd66e844,140542,Ejerlejlighed,0,None,None,0,0101,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:21:38.990995+02:00,None,Ejendomsbeliggenhedsregister,0
72279,89731,http://data.gov.dk/Ejendomsbeliggenhedsregistret,676ecb69-56b8-42a7-8e13-eecf656295ad,255877,Ejerlejlighed,0,None,None,0,0219,...,konverteretFraESR,52.20.05,konverteretFraESR,0001-01-01T15:00:00.000000+00:50,None,Geodatastyrelsen,2019-04-28T13:22:08.072299+02:00,None,Ejendomsbeliggenhedsregister,0


In [14]:
ebrsql_df['part'] = (ebrsql_df.index % 10) + 1


In [ ]:
ebrsql_df

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

# MySQL connection setup
username = 'root'
password = '1234'
host = 'localhost'
database = 'bolig'

# Establish a connection to the MySQL database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

# SQL query to select all data from the "ebr" table
query = 'SELECT * FROM ebr'

# Load the data into a pandas DataFrame
df = pd.read_sql_query(query, engine)

# Close the database connection
engine.dispose()

# Calculate the percentage distribution for each kommuneinddelingKommunekode
percentage_distribution = df['kommuneinddelingKommunekode'].value_counts(normalize=True)

# Split the dataset into two equal parts using stratified sampling
df_part1, df_part2 = train_test_split(
    df,
    test_size=0.5,
    stratify=df['kommuneinddelingKommunekode'],
    random_state=42
)

# Now you have your original dataset "df" and two equal halves "df_part1" and "df_part2"
# with the same percentage distribution for the kommuneinddelingKommunekode column.


In [6]:
# Shuffle the two parts independently to randomize the order within each group
df_part1_shuffled = df_part1.sample(frac=1, random_state=42).reset_index(drop=True)
df_part2_shuffled = df_part2.sample(frac=1, random_state=42).reset_index(drop=True)


In [7]:
len(df_part1_shuffled)

89731

In [9]:
import pandas as pd
import numpy as np

# Assuming you already have the two equal parts "df_part1_shuffled" and "df_part2_shuffled"
# obtained through the previous steps

# Concatenate the two parts to work with the entire dataset
df_combined = pd.concat([df_part1_shuffled, df_part2_shuffled], ignore_index=True)

# Calculate the total number of rows and the target number of rows for each part (close to 25%)
total_rows = len(df_combined)
target_part_size = int(total_rows * 0.25)

# Create an array of numbers from 1 to 4, where each number appears target_part_size times
part_numbers = np.repeat(np.arange(1, 5), target_part_size)

# Adjust the length of the part_numbers array to match the total number of rows
part_numbers = np.resize(part_numbers, total_rows)

# Randomly shuffle the array to ensure random distribution
np.random.shuffle(part_numbers)

# Assign the part_numbers array as the "part" column to df_combined
df_combined['part'] = part_numbers

# Split df_combined back into the two parts
df_part1_shuffled_with_part = df_combined[df_combined['part'].isin([1, 2])].reset_index(drop=True)
df_part2_shuffled_with_part = df_combined[df_combined['part'].isin([3, 4])].reset_index(drop=True)

# Now, df_part1_shuffled_with_part and df_part2_shuffled_with_part represent the two equal halves
# with the same percentage distribution for the kommuneinddelingKommunekode column,
# both parts are shuffled randomly, and each part has the "part" column with values ranging from 1 to 4
# where each number appears close to 25% of the total rows.


In [11]:
len(df_part1_shuffled_with_part)

89733

In [17]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, types
from math import ceil

# MySQL connection setup
username = 'root'
password = '1234'
host = 'localhost'
database = 'bolig'

engine_str = f"mysql+mysqlconnector://{username}:{password}@{host}/{database}"
engine = create_engine(engine_str)

# Fetching the dtypes for each column
dtype_dict = ebrsql_df.dtypes.map(lambda x: types.TEXT 
                                  if x == "object" 
                                  else types.Float(precision=3, asdecimal=True) 
                                  if x == "float" 
                                  else types.Integer()).to_dict()

# Create table if doesn't exist
ebrsql_df[:0].to_sql('ebr_sample10', con=engine, if_exists='replace', dtype=dtype_dict)

# Upload data in chunks
n = 1000  # chunk row size
total_chunks = ceil(len(ebrsql_df) / n)

for i in range(total_chunks):
    print(f'Uploading chunk {i+1}/{total_chunks}...')
    ebrsql_df[i*n:(i+1)*n].to_sql('ebr_sample10', con=engine, if_exists='append', dtype=dtype_dict)
    
print("Upload complete.")


Uploading chunk 1/73...
Uploading chunk 2/73...
Uploading chunk 3/73...
Uploading chunk 4/73...
Uploading chunk 5/73...
Uploading chunk 6/73...
Uploading chunk 7/73...
Uploading chunk 8/73...
Uploading chunk 9/73...
Uploading chunk 10/73...
Uploading chunk 11/73...
Uploading chunk 12/73...
Uploading chunk 13/73...
Uploading chunk 14/73...
Uploading chunk 15/73...
Uploading chunk 16/73...
Uploading chunk 17/73...
Uploading chunk 18/73...
Uploading chunk 19/73...
Uploading chunk 20/73...
Uploading chunk 21/73...
Uploading chunk 22/73...
Uploading chunk 23/73...
Uploading chunk 24/73...
Uploading chunk 25/73...
Uploading chunk 26/73...
Uploading chunk 27/73...
Uploading chunk 28/73...
Uploading chunk 29/73...
Uploading chunk 30/73...
Uploading chunk 31/73...
Uploading chunk 32/73...
Uploading chunk 33/73...
Uploading chunk 34/73...
Uploading chunk 35/73...
Uploading chunk 36/73...
Uploading chunk 37/73...
Uploading chunk 38/73...
Uploading chunk 39/73...
Uploading chunk 40/73...
Uploading